In [1]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
from fastlane_bot.modes import triangle_single_bancor3
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.10.1 (07/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require


C:\Users\Kveen\AppData\Local\Temp\ipykernel_36012\3226121854.py:27: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.2 [requirements >= 3.0 is met]


In [2]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET)
assert (C.NETWORK == C.NETWORK_MAINNET)
assert (C.PROVIDER == C.PROVIDER_ALCHEMY)
setup_bot = CarbonBot(ConfigObj=C)

In [3]:


pools = None
with open('fastlane_bot/data/tests/latest_pool_data_testing.json') as f:
    pools = json.load(f)
pools = [pool for pool in pools]

In [4]:
pools[0]
static_pools = pools

In [5]:
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

In [6]:
static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
        
tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,bancor_v3,uniswap_v3,uniswap_v2,sushiswap_v2"

exchanges = exchanges.split(",")
arb_mode = "multi"

alchemy_max_block_fetch = 20

In [7]:
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]


In [8]:
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)

In [9]:
static_pool_data['exchange_name'].unique()

array(['uniswap_v3', 'bancor_v3', 'uniswap_v2', 'sushiswap_v2'],
      dtype=object)

In [10]:
from joblib import Parallel, delayed

# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"

2023-07-17 16:16:33,911 [fastlane:INFO] - Time taken to add initial pools: 0.09121036529541016


In [11]:
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot

In [12]:
bot = init_bot(mgr)

2023-07-17 16:16:33,948 [fastlane:INFO] - Initializing the bot...


In [13]:
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()

2023-07-17 16:16:34,138 [fastlane:INFO] - Removed 3242 unmapped uniswap_v2/sushi pools. 1897 uniswap_v2/sushi pools remaining
2023-07-17 16:16:34,138 [fastlane:INFO] - Unmapped uniswap_v2/sushi pools:
2023-07-17 16:16:34,299 [fastlane:INFO] - uniswap_v3: 0
2023-07-17 16:16:34,300 [fastlane:INFO] - uniswap_v2: 3242
2023-07-17 16:16:34,301 [fastlane:INFO] - sushiswap_v2: 0
2023-07-17 16:16:34,302 [fastlane:INFO] - uniswap_v3: 636
2023-07-17 16:16:34,303 [fastlane:INFO] - sushiswap_v2: 78
2023-07-17 16:16:34,304 [fastlane:INFO] - uniswap_v2: 0
2023-07-17 16:16:34,304 [fastlane:INFO] - bancor_v2: 0
2023-07-17 16:16:34,305 [fastlane:INFO] - bancor_v3: 34
2023-07-17 16:16:34,305 [fastlane:INFO] - carbon_v1: 220
2023-07-17 16:16:34,328 [fastlane:INFO] - uniswap_v3_zero_liquidity_pools: 858
2023-07-17 16:16:34,329 [fastlane:INFO] - sushiswap_v2_zero_liquidity_pools: 34
2023-07-17 16:16:34,330 [fastlane:INFO] - uniswap_v2_zero_liquidity_pools: 0
2023-07-17 16:16:34,330 [fastlane:INFO] - bancor_

In [14]:
tokens = bot.db.get_tokens()

In [15]:
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}

In [16]:
flashloan_tokens = bot.setup_flashloan_tokens(None)
CCm = bot.setup_CCm(None)
pools = db.get_pool_data_with_tokens()

In [18]:
arb_finder = bot._get_arb_finder("multi")
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=False,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()
(
            best_profit,
            best_trade_instructions_df,
            best_trade_instructions_dic,
            best_src_token,
            best_trade_instructions,
        ) = r

In [20]:
best_trade_instructions_dic

[{'cid': '0xcb487fc19b1e23857870b0743650419594c1883e4cee744984589f0093c6366a',
  'tknin': 'agEUR-Bce8',
  'amtin': 3290.0749664902687,
  'tknout': 'USDC-eB48',
  'amtout': -3623.1383405923843,
  'error': None},
 {'cid': '21778071482940061661655974875633165533369-1',
  'tknin': 'USDC-eB48',
  'amtin': 3611.2638695905916,
  'tknout': 'agEUR-Bce8',
  'amtout': -3290.074966498185,
  'error': None}]

In [21]:
(
ordered_trade_instructions_dct,
tx_in_count,
) = bot._simple_ordering_by_src_token(
best_trade_instructions_dic, best_src_token
)

In [22]:
ordered_scaled_dcts = bot._basic_scaling(
            ordered_trade_instructions_dct, best_src_token
        )

In [24]:
ordered_scaled_dcts

[{'cid': '21778071482940061661655974875633165533369-1',
  'tknin': 'USDC-eB48',
  'amtin': 3607.6526057210012,
  'tknout': 'agEUR-Bce8',
  'amtout': -3290.074966498185,
  'error': None},
 {'cid': '0xcb487fc19b1e23857870b0743650419594c1883e4cee744984589f0093c6366a',
  'tknin': 'agEUR-Bce8',
  'amtin': 3290.0749664902687,
  'tknout': 'USDC-eB48',
  'amtout': -3623.1383405923843,
  'error': None}]

In [28]:
print({bot.db is not None})

{True}


In [26]:
# Convert the trade instructions
ordered_trade_instructions_objects = bot._convert_trade_instructions(ordered_scaled_dcts)

AttributeError: 'NoneType' object has no attribute 'exchange_name'

In [ ]:

tx_route_handler = bot.TxRouteHandlerClass(
            trade_instructions=ordered_trade_instructions_objects
        )

In [ ]:
agg_trade_instructions = (
            tx_route_handler.aggregate_carbon_trades(ordered_trade_instructions_objects)
            if bot._carbon_in_trade_route(ordered_trade_instructions_objects)
            else ordered_trade_instructions_objects
        )

In [19]:




# Calculate the trade instructions
calculated_trade_instructions = tx_route_handler.calculate_trade_outputs(
    agg_trade_instructions
)

AttributeError: 'NoneType' object has no attribute 'exchange_name'

In [ ]:
ordered_scaled_dcts = bot._basic_scaling(
            ordered_trade_instructions_dct, best_src_token
        )
# Convert the trade instructions
ordered_trade_instructions_objects = bot._convert_trade_instructions(
    ordered_scaled_dcts)
tx_route_handler = bot.TxRouteHandlerClass(
            trade_instructions=ordered_trade_instructions_objects
        )
agg_trade_instructions = (
            tx_route_handler.aggregate_carbon_trades(ordered_trade_instructions_objects)
            if bot._carbon_in_trade_route(ordered_trade_instructions_objects)
            else ordered_trade_instructions_objects
        )
# Calculate the trade instructions
calculated_trade_instructions = tx_route_handler.calculate_trade_outputs(
    agg_trade_instructions
)

In [ ]:
trade_instructions = [{'cid': '0x38a4847880387ca4a93a5d202263135d162f37adf6e46b505f1f2117f4909a83', 'tknin': 'WETH-6Cc2', 'amtin': 4.013580159749836, 'tknout': 'USDC-eB48', 'amtout': -7999.4359412789345, 'error': None}, {'cid': '1701411834604692317316873037158841057299-1', 'tknin': 'USDC-eB48', 'amtin': 45.506197613396125, 'tknout': 'WETH-6Cc2', 'amtout': -0.023142650372126816, 'error': None}, {'cid': '1701411834604692317316873037158841057337-1', 'tknin': 'USDC-eB48', 'amtin': 121.86471483280911, 'tknout': 'WETH-6Cc2', 'amtout': -0.061423963598862485, 'error': None}, {'cid': '1701411834604692317316873037158841057343-1', 'tknin': 'USDC-eB48', 'amtin': 1990.0, 'tknout': 'WETH-6Cc2', 'amtout': -1.0, 'error': None}, {'cid': '1701411834604692317316873037158841057358-1', 'tknin': 'USDC-eB48', 'amtin': 3.112472768407315e-08, 'tknout': 'WETH-6Cc2', 'amtout': -1.596145438043095e-11, 'error': None}, {'cid': '1701411834604692317316873037158841057382-1', 'tknin': 'USDC-eB48', 'amtin': 37.481971740722656, 'tknout': 'WETH-6Cc2', 'amtout': -0.019727351143956184, 'error': None}, {'cid': '1701411834604692317316873037158841057386-1', 'tknin': 'USDC-eB48', 'amtin': 3.6403071135282516e-07, 'tknout': 'WETH-6Cc2', 'amtout': -1.9160495412506862e-10, 'error': None}, {'cid': '1701411834604692317316873037158841057405-1', 'tknin': 'USDC-eB48', 'amtin': 3.525674401316792e-07, 'tknout': 'WETH-6Cc2', 'amtout': -1.836291119161615e-10, 'error': None}, {'cid': '1701411834604692317316873037158841057409-1', 'tknin': 'USDC-eB48', 'amtin': 0.8508408926427364, 'tknout': 'WETH-6Cc2', 'amtout': -0.00047268935850297567, 'error': None}, {'cid': '1701411834604692317316873037158841057569-1', 'tknin': 'USDC-eB48', 'amtin': 50.3521728515625, 'tknout': 'WETH-6Cc2', 'amtout': -0.026515096426010132, 'error': None}, {'cid': '1701411834604692317316873037158841057432-1', 'tknin': 'USDC-eB48', 'amtin': 823.4780578613281, 'tknout': 'WETH-6Cc2', 'amtout': -0.4148503690958023, 'error': None}, {'cid': '1701411834604692317316873037158841057434-1', 'tknin': 'USDC-eB48', 'amtin': 0.30484674870967865, 'tknout': 'WETH-6Cc2', 'amtout': -0.0001604456572295021, 'error': None}, {'cid': '1701411834604692317316873037158841057441-1', 'tknin': 'USDC-eB48', 'amtin': 929.7200231026363, 'tknout': 'WETH-6Cc2', 'amtout': -0.4728686694506905, 'error': None}, {'cid': '1701411834604692317316873037158841057444-1', 'tknin': 'USDC-eB48', 'amtin': 8.926726877689362e-07, 'tknout': 'WETH-6Cc2', 'amtout': -4.69867700303439e-10, 'error': None}, {'cid': '1701411834604692317316873037158841057458-1', 'tknin': 'USDC-eB48', 'amtin': 7.694998203078285e-07, 'tknout': 'WETH-6Cc2', 'amtout': -4.228022376651097e-10, 'error': None}, {'cid': '1701411834604692317316873037158841057482-1', 'tknin': 'USDC-eB48', 'amtin': 1.3181804613006764e-13, 'tknout': 'WETH-6Cc2', 'amtout': -6.773942102560546e-17, 'error': None}, {'cid': '1701411834604692317316873037158841057484-1', 'tknin': 'USDC-eB48', 'amtin': 1073.2558631896973, 'tknout': 'WETH-6Cc2', 'amtout': -0.5420484151691198, 'error': None}, {'cid': '1701411834604692317316873037158841057499-1', 'tknin': 'USDC-eB48', 'amtin': 30.675735473632812, 'tknout': 'WETH-6Cc2', 'amtout': -0.016018658876419067, 'error': None}, {'cid': '1701411834604692317316873037158841057513-1', 'tknin': 'USDC-eB48', 'amtin': 951.2761421203613, 'tknout': 'WETH-6Cc2', 'amtout': -0.4804424960166216, 'error': None}, {'cid': '1701411834604692317316873037158841057528-1', 'tknin': 'USDC-eB48', 'amtin': 1329.2054503213294, 'tknout': 'WETH-6Cc2', 'amtout': -0.6738792466624943, 'error': None}, {'cid': '1701411834604692317316873037158841057545-1', 'tknin': 'USDC-eB48', 'amtin': 615.4639225006104, 'tknout': 'WETH-6Cc2', 'amtout': -0.3092783526517451, 'error': None}]

In [ ]:
tx_route_handler = bot.TxRouteHandlerClass(
            trade_instructions=ordered_trade_instructions_objects
        )

In [ ]:
encoded_trade_instructions = tx_route_handler.custom_data_encoder(
            agg_trade_instructions
        )